# Setup

In [ ]:
!apt-get update
!apt-get install git

!pip install scikit-learn===0.24.0 #needed for autopytorch but need >=1.0 needed for handcrafted feature generation or get https://github.com/mne-tools/mne-features/issues/73
#!pip install scikit-learn

# MNE-BIDS preprocessing functions
!pip install mne
!pip install MNE-bids
#!pip install -r https://raw.githubusercontent.com/mne-tools/mne-bids-pipeline/main/requirements.txt
!pip install typing_extensions psutil joblib dask[distributed] jupyter-server-proxy scikit-learn pandas seaborn json_tricks coloredlogs python-picard fire pyqt5 pyvista pyvistaqt openpyxl

!git clone https://github.com/mne-tools/mne-bids-pipeline.git /cloned-mne-bids-pipeline

!pip install AutoReject
!pip install coffeine
!pip install Braindecode

#!rm -r /cloned-paper-repo
!git clone https://github.com/meeg-ml-benchmarks/brain-age-benchmark-paper.git /cloned-paper-repo

!pip install -U matplotlib

!pip install -U scipy
!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html

!pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas



### Restart Runtime 

make sure all updated packages are loaded in their newest versions


In [ ]:
import os
def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

###Mount Google Drive for access to data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
######################################################################################################
#  ________   _________   ___    ___ ___  ___  ________  ________  ___  __      _______   _______
# |\   ___  \|\___   ___\|\  \  /  /|\  \|\  \|\   __  \|\   ____\|\  \|\  \   /  ___  \ /  ___  \
# \ \  \\ \  \|___ \  \_|\ \  \/  / | \  \\\  \ \  \|\  \ \  \___|\ \  \/  /|_/__/|_/  //__/|_/  /|
#  \ \  \\ \  \   \ \  \  \ \    / / \ \   __  \ \   __  \ \  \    \ \   ___  \__|//  / /__|//  / /
#   \ \  \\ \  \   \ \  \  /     \/   \ \  \ \  \ \  \ \  \ \  \____\ \  \\ \  \  /  /_/__  /  /_/__
#    \ \__\\ \__\   \ \__\/  /\   \    \ \__\ \__\ \__\ \__\ \_______\ \__\\ \__\|\________\\________\
#     \|__| \|__|    \|__/__/ /\ __\    \|__|\|__|\|__|\|__|\|_______|\|__| \|__| \|_______|\|_______|
#                        |__|/ \|__|
######################################################################################################

import mne
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge


# Import Preprocessed

In [ ]:
import pickle
%cd /content/drive/My Drive/brainAge_competition/
with open(r"finalStage_test_preproc.sav", "rb") as input_file:
  finalStage_test_preproc = pickle.load(input_file)

/content/drive/My Drive/brainAge_competition


# Pick chans 

In [ ]:

chans_22subset = ['E36', 'E104', 'E24', 'E124', 'E33', 'E122', 'E22', 'E9', 
         'E14', 'E21', 'E15', 'E11', 'E70', 'E83', 'E52', 'E92', 'E58', 
         'E96', 'E45', 'E108', 'E72', 'E62']

for r in finalStage_test_preproc:
  r.pick_channels(chans_22subset)


# epoch

In [ ]:
events = mne.make_fixed_length_events(
            finalStage_test_preproc[0], 
            #id=3000, 
            start=2,
            #duration=rest_epochs_duration,
            duration = 2 - 1 / finalStage_test_preproc[0].info['sfreq'],
            #overlap=rest_epochs_overlap,
            overlap = 0.,
            stop=38,
            )


finalStage_testepoched_data = [None] * len(finalStage_test_preproc)

for i in range(len(finalStage_test_preproc)):
  print(i)
  inputdat =  finalStage_test_preproc[i]
  finalStage_testepoched_data[i] = mne.Epochs(inputdat, events=events, event_id=1,
                      tmin=0, tmax=2,
                      proj=False, 
                      baseline=None,
                      preload=True, 
                      #decim=decim,
                      #metadata=metadata,
                      #event_repeated=event_repeated,
                      reject=None
                      )


(18, 22, 501)

# auto reject 

In [ ]:
import autoreject
for i in range(len(finalStage_testepoched_data)):
  print(f"#########   PTP RUNNING IS {i}   ################")
  ar = autoreject.AutoReject(n_jobs=4, cv=5, verbose = False) # verbose=False b/c the output is so plentiful that colab has a hard time with it 
  finalStage_testepoched_data[i] = ar.fit_transform(finalStage_testepoched_data[i])


import pickle
%cd /content/drive/My Drive/brainAge_competition/
filename = 'finalStage_testSet_22chans_2secEpochs_autoreject.sav' 
pickle.dump(finalStage_testepoched_data, open(filename, 'wb'))


# Gen welch psds

In [ ]:

################### test set ##############################
import pickle
%cd /content/drive/My Drive/brainAge_competition/
with open(r"finalStage_testSet_22chans_2secEpochs_autoreject.sav", "rb") as input_file:
  finalStage_testepoched_data = pickle.load(input_file)

finalStage_testout = [None] * len(finalStage_testepoched_data)
for i in range(len(finalStage_testepoched_data)):
  spectrum = finalStage_testepoched_data[i].compute_psd(method="multitaper", fmin=0.9, fmax=40) 
  finalStage_testout[i]=spectrum.get_data(return_freqs=True)


(16, 22, 79)

# Fooof setup

In [ ]:
!pip install fooof

 # Import the FOOOF object
from fooof import FOOOF
from joblib.parallel import Parallel, delayed
# Imports for helping with managing our simulated data and results
import os
import numpy as np

# Import the FOOOFGroup object
from fooof import FOOOFGroup

# Import utilities for working with FOOOF objects
from fooof.objs import fit_fooof_3d, combine_fooofs

# Import simulation & IO utilities to help with the example
from fooof.sim.gen import gen_freqs, gen_group_power_spectra
from fooof.sim.params import param_sampler
from fooof.utils.io import load_fooofgroup

# dictionary for storing output
class fooofResultsDictionaries(dict):
 
  # __init__ function
  def __init__(self):
    self = dict()
 
  # Function to add key:value
  def add(self, key, value):
    self[key] = value

# run fooofs and save to dictionary

In [ ]:
finalStage_testfooof_welch_psd = finalStage_testout

finalStage_testfooofDictResults_avgOverTrialsThenChannels = fooofResultsDictionaries()
finalStage_testfooofDictResults_avgOverTrials = fooofResultsDictionaries()

chanList = finalStage_testepoched_data[0].ch_names

for ptp in range(len(finalStage_testfooof_welch_psd)):
  if not isinstance(finalStage_testfooof_welch_psd[ptp], str):
      print(ptp)
      avgOverTrials = np.mean(finalStage_testfooof_welch_psd[ptp][0], axis=0) #61 x 64
      avgOverTrialsThenChannels = np.mean(avgOverTrials, axis=0) #64

      freqs = finalStage_testfooof_welch_psd[0][1]
      freq_range=[0.9, 40]

      if ptp == 147:
        fg = FOOOFGroup(
                        peak_width_limits=[1,12],# [1.5, 8], 
                        min_peak_height=0.05, 
                        max_n_peaks=6
                        )
        fg.fit(freqs, avgOverTrials, freq_range)
      else:
        fg = FOOOFGroup(
                        peak_width_limits=[1.5, 8], 
                        min_peak_height=0.05, 
                        max_n_peaks=6
                        )
        fg.fit(freqs, avgOverTrials, freq_range)

      for electrode in range(len(chanList)):

        finalStage_testfooofDictResults_avgOverTrials.add(f'ptp_{ptp}_Electrode{chanList[electrode]}', fg.get_fooof(ind=electrode))

  else: 
    print(f"no data for {sub}")


# Save ###############################
%cd '/content/drive/My Drive/brainAge_competition/'
with open("finalStage_testSet_fooofResults_avgOverTrials_22chans_2secEpochs_autoreject", "wb") as fp:   #Pickling
  pickle.dump(finalStage_testfooofDictResults_avgOverTrials, fp)


False


# Reformat fooofs to array for auto-pytorch

In [ ]:


# combine fooof features into array ###################

#periodic features first#####

finalStage_testfooofs_periodic = np.empty([400, 79*22]) # freq data points by channels

for ptp in range(0,400,1):
  placeholder = np.empty(0)
  for electrode in range(len(chanList)):
    placeholder = np.append(placeholder,finalStage_testfooofDictResults_avgOverTrials[f'ptp_{ptp}_Electrode{chanList[electrode]}'].fooofed_spectrum_)
  finalStage_testfooofs_periodic[ptp,:] = placeholder

# any nan? #
np.where(np.isnan(finalStage_testfooofs_periodic)) 

freqNames = finalStage_testfooofDictResults_avgOverTrials[f'ptp_{ptp}_Electrode{chanList[electrode]}'].freqs
freqNames = np.arange(1,40.5,0.5)
COLNAMES = list()
for electrode in range(len(chanList)):
  electrodeColNames = list()
  for freq in range(len(freqNames)):
    colname = f"{chanList[electrode]}_{freqNames[freq]}"
    electrodeColNames = np.append(electrodeColNames,colname)
  COLNAMES = np.append(COLNAMES, electrodeColNames)

finalStage_testdf_periodic = pd.DataFrame(finalStage_testfooofs_periodic, columns = COLNAMES)

# aperioid features second #####

finalStage_testnonperiodics_ai4healthFormat = np.empty([400, 2*22]) # freq data points by channels

for ptp in range(0,400,1):
  placeholder = np.empty(0)
  for electrode in range(len(chanList)):
    placeholder = np.append(placeholder,finalStage_testfooofDictResults_avgOverTrials[f'ptp_{ptp}_Electrode{chanList[electrode]}'].aperiodic_params_)
  finalStage_testnonperiodics_ai4healthFormat[ptp,:] = placeholder

aperiodic_params = ["y_intercept", "slope"]

aperiodicCOLNAMES = list()
for electrode in range(len(chanList)):
  aperiodicelectrodeColNames = list()
  for param in range(len(aperiodic_params)):
    colname = f"{chanList[electrode]}_{aperiodic_params[param]}"
    aperiodicelectrodeColNames = np.append(aperiodicelectrodeColNames,colname)
  aperiodicCOLNAMES = np.append(aperiodicCOLNAMES, aperiodicelectrodeColNames)

finalStage_testaperiodic_df = pd.DataFrame(finalStage_testnonperiodics_ai4healthFormat, columns = aperiodicCOLNAMES)

finalStage_testset_fooofs = pd.merge(finalStage_testdf_periodic, finalStage_testaperiodic_df, left_index=True, right_index=True)

np.where(np.isnan(finalStage_testset_fooofs)) # any nans?

# save array'd fooofs
%cd '/content/drive/My Drive/brainAge_competition/'
with open("finalStage_testset_fooofs", "wb") as fp:   #Pickling
  pickle.dump(finalStage_testset_fooofs, fp)